In [ ]:
pip install sentence.transformers groq faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.9/134.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 20.5 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
from sentence_transformers import SentenceTransformer
from groq import Groq
import faiss
import json
from google.colab import userdata
import requests

""" **Step 2: Initialize Groq Client** """
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
llm_model = "llama-3.3-70b-versatile"

# Initialize Groq client
client = Groq(api_key=GROQ_API_KEY)

""" **Step 3: Load the Embedding Model** """
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embedding_dim = embedding_model.get_sentence_embedding_dimension()  # typically 384

""" **Step 4: Prepare Documents** """
documents = [
    "The capital of France is Paris.",
    "Python is a popular programming language.",
    "Machine learning is a subset of artificial intelligence.",
    "The Eiffel Tower is in Paris.",
    "Groq provides fast AI inference."
]

""" **Step 5: Generate Embeddings for Documents** """
doc_embeddings = embedding_model.encode(documents)
doc_embeddings = np.array(doc_embeddings).astype('float32')  # FAISS requires float32

""" **Step 6: Set Up FAISS Vector Database** """
index = faiss.IndexFlatL2(embedding_dim)  # L2 distance metric
index.add(doc_embeddings)  # Add document embeddings to the index

""" **Step 7: Define the Retrieval Function** """
def retrieve_documents(query, top_k=3):
    # Embed the query
    query_embedding = embedding_model.encode([query])
    query_embedding = np.array(query_embedding).astype('float32')

    # Search in FAISS
    distances, indices = index.search(query_embedding, top_k)

    # Retrieve the matching documents
    retrieved_docs = [documents[i] for i in indices[0]]
    return retrieved_docs

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def generate_response(query, retrieved_docs, temperature=0.4):
    # Create context from retrieved documents
    context = "\n".join([f"- {doc}" for doc in retrieved_docs])

    prompt = f"""
    You are a concise and accurate assistant. Use the provided context to answer the query directly and clearly.
    If the context doesn't contain relevant information, then simply say **Can't provide a valid ans**.

    Context:
    {context}

    Query: {query}

    Answer:"""

    # ✅ Make sure indentation here is only 4 spaces (aligned with `prompt`)
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }

    data = {
        "model": llm_model,
        "messages": [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        "temperature": temperature
    }

    try:
        response = requests.post(API_URL, headers=headers, data=json.dumps(data))
        if response.status_code == 200:
            response_data = response.json()
            return response_data['choices'][0]['message']['content'].strip()
        else:
            return f"Error: API returned status code {response.status_code}: {response.text}"
    except requests.RequestException as e:
        return f"Error in API request: {str(e)}"

In [ ]:
API_URL = "https://api.groq.com/openai/v1/chat/completions"

In [ ]:
documents = [
    "The capital of France is Paris.",
    "Python is a popular programming language.",
    "Machine learning is a subset of artificial intelligence.",
    "The Eiffel Tower is in Paris.",
    "Groq provides fast AI inference.",
    "The human heart has four chambers.",
    "Photosynthesis is the process plants use to convert light energy into chemical energy.",
    "The internet is a global network of interconnected computer networks.",
    "The Mona Lisa is a portrait painting by Leonardo da Vinci.",
    "Mount Everest is the Earth's highest mountain above sea level.",
    "Quantum computing is a type of computation that uses quantum phenomena.",
    "The Roman Empire was a vast empire that controlled much of Europe and the Mediterranean.",
    "Shakespeare was an English playwright, poet, and actor.",
    "The speed of light in a vacuum is approximately 299,792 kilometers per second."
]

In [ ]:
index = faiss.IndexFlatL2(embedding_dim)
index.add(doc_embeddings)

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/Django_(web_framework)"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

documents = [p.get_text() for p in soup.find_all("p")]